In [88]:
#Latihan1, Tugas1
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}),
    
    # Menambahkan course baru
    InsertOne({'course': 'Japanese 203', 'enrollments': 20, 'department': 'Japanese'}),

    # Memperbarui course History 201
    UpdateOne(
        {'course': 'CS 102'},
        {'$set': {'enrollments': 35}}
    ),

    # Menghapus pelajaran yang enrollmentnya kurang dari 20
    DeleteOne({'enrollments': {'$lt': 20}})
]

result = courses_collection.bulk_write(operations)
print("Bulk write successfully.")
print(f"Document Inserted: {result.inserted_count}")
print(f"Document Updated: {result.modified_count}")
print(f"Document Deleted: {result.deleted_count}")

print("Data for all current course: ")
for course in courses_collection.find():
    print(course)

Bulk write successfully.
Document Inserted: 5
Document Updated: 1
Document Deleted: 1
Data for all current course: 
{'_id': ObjectId('693424093bcf340c5c3379d1'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('693424093bcf340c5c3379d2'), 'course': 'CS 102', 'enrollments': 35, 'department': 'Computer Science'}
{'_id': ObjectId('693424093bcf340c5c3379d3'), 'course': 'History 201', 'enrollments': 20, 'department': 'History'}
{'_id': ObjectId('693424093bcf340c5c3379d5'), 'course': 'Japanese 203', 'enrollments': 20, 'department': 'Japanese'}


In [89]:
#Latihan2
# Query for courses with enrollments greater than 20
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)

{'_id': ObjectId('693424093bcf340c5c3379d1'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('693424093bcf340c5c3379d2'), 'course': 'CS 102', 'enrollments': 35, 'department': 'Computer Science'}


In [90]:
#Tugas2
# Query courses in Computer Science or Mathematics departments
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)

{'_id': ObjectId('693424093bcf340c5c3379d1'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('693424093bcf340c5c3379d2'), 'course': 'CS 102', 'enrollments': 35, 'department': 'Computer Science'}


In [91]:
#Latihan3
# Average enrollment per department using aggregation
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Japanese', 'average_enrollment': 20.0}
{'_id': 'Computer Science', 'average_enrollment': 35.0}
{'_id': 'Mathematics', 'average_enrollment': 30.0}
{'_id': 'History', 'average_enrollment': 20.0}


In [92]:
#Tugas3
# Maximum enrollment per department
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Japanese', 'max_enrollment': 20}
{'_id': 'Computer Science', 'max_enrollment': 35}
{'_id': 'Mathematics', 'max_enrollment': 30}
{'_id': 'History', 'max_enrollment': 20}


In [93]:
#Latihan4
# Projection to rename fields
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('693424093bcf340c5c3379d1'), 'enrollments': 30, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'_id': ObjectId('693424093bcf340c5c3379d2'), 'enrollments': 35, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'_id': ObjectId('693424093bcf340c5c3379d3'), 'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}
{'_id': ObjectId('693424093bcf340c5c3379d5'), 'enrollments': 20, 'course_name': 'Japanese 203', 'department_name': 'Japanese'}


In [94]:
#Tugas4
# Adding enrollment category field based on enrollments
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('693424093bcf340c5c3379d1'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('693424093bcf340c5c3379d2'), 'course': 'CS 102', 'enrollments': 35, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('693424093bcf340c5c3379d3'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}
{'_id': ObjectId('693424093bcf340c5c3379d5'), 'course': 'Japanese 203', 'enrollments': 20, 'department': 'Japanese', 'enrollment_category': 'low'}


In [95]:
#Homework1
pipeline = [
    {'$group': {'_id': '$department', 'course_count': {'$sum': 1}}}
]

print("A count of courses per department:")
for result in courses_collection.aggregate(pipeline):
    print(result)

A count of courses per department:
{'_id': 'Computer Science', 'course_count': 1}
{'_id': 'Mathematics', 'course_count': 1}
{'_id': 'History', 'course_count': 1}
{'_id': 'Japanese', 'course_count': 1}


In [97]:
#Homework2
# Use $match and $group together to filter and get only courses with enrollments over 25 in 'Computer Science'
pipeline = [
    {'$match': {'department': 'Computer Science', 'enrollments': {'$gt': 25}}},
    {'$group': {'_id': '$department', 'total_courses': {'$sum': 1}, 'avg_enrollments': {'$avg': '$enrollments'}}}
]

for result in courses_collection.aggregate(pipeline):
    print(f"Department: {result['_id']}, Total Courses: {result['total_courses']}, Average Enrollments: {result['avg_enrollments']:.2f}")

Department: Computer Science, Total Courses: 1, Average Enrollments: 35.00


In [100]:
#Homework3.1
# collection students
from pymongo import MongoClient, InsertOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
students_collection = db['students']

add = [
    InsertOne({'student_id': 1, 'name': 'Asep', 'course': 'CS 102'}),
    InsertOne({'student_id': 2, 'name': 'Nahmier', 'course': 'History 201'}),
    InsertOne({'student_id': 3, 'name': 'Jane', 'course': 'Math 101'}),
    InsertOne({'student_id': 4, 'name': 'Julio', 'course': 'CS 102'}),
    InsertOne({'student_id': 5, 'name': 'Flip', 'course': 'Japanese 203'})
]

result = students_collection.bulk_write(add)

print("Students collection created successfully.")
print(f"Dokumen disisipkan: {result.inserted_count}")
print("\nData on all students:")
for student in students_collection.find():
    print(student)

Students collection created successfully.
Dokumen disisipkan: 5

Data on all students:
{'_id': ObjectId('69342aa43bcf340c5c3379dd'), 'student_id': 1, 'name': 'Asep', 'course': 'CS 102'}
{'_id': ObjectId('69342aa43bcf340c5c3379de'), 'student_id': 2, 'name': 'Nahmier', 'course': 'History 201'}
{'_id': ObjectId('69342aa43bcf340c5c3379df'), 'student_id': 3, 'name': 'Jane', 'course': 'Math 101'}
{'_id': ObjectId('69342aa43bcf340c5c3379e0'), 'student_id': 4, 'name': 'Julio', 'course': 'CS 102'}
{'_id': ObjectId('69342aa43bcf340c5c3379e1'), 'student_id': 5, 'name': 'Flip', 'course': 'Japanese 203'}


In [102]:
#Homework3.2
pipeline = [
    {
        '$lookup': {
            'from': 'students',       
            'localField': 'course',    
            'foreignField': 'course', 
            'as': 'student_list'      
        }
    },
    {
        '$project': {                 
            '_id': 0,
            'course': 1,
            'department': 1,
            'enrollments': 1,
            'student_list.name': 1
        }
    }
]

# Menjalankan Agregasi
print("Result:\n")
for result in courses_collection.aggregate(pipeline):
    print(result)

Result:

{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_list': [{'name': 'Jane'}]}
{'course': 'CS 102', 'enrollments': 35, 'department': 'Computer Science', 'student_list': [{'name': 'Asep'}, {'name': 'Julio'}]}
{'course': 'History 201', 'enrollments': 20, 'department': 'History', 'student_list': [{'name': 'Nahmier'}]}
{'course': 'Japanese 203', 'enrollments': 20, 'department': 'Japanese', 'student_list': [{'name': 'Flip'}]}
